In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from adtxns_utils import *

In [2]:
# Define a wrapper function for saving figures with bbox_inches='tight'
def save_figure(fig, filename):
    fig.savefig(filename, bbox_inches='tight')


# fig_path = '/home/claudio/tesi/manuscript/final_figures'
fig_path = '/home/claudio/tesi/Template2/final_figures'
# fig_path = './wait'

save = False
put_title = True

# Sarafu handling


In [3]:
txns_fn = '/home/claudio/tesi/sarafu/final_data/00_tx_with_bal_bwd_allpositivebalances.csv'
TXDF = pd.read_csv(txns_fn,sep=",",header=0,dtype={'id': 'int',
                                                    'date': 'string',
                                                    'type': 'category',
                                                    'source_xdai': 'string',
                                                    'target_xdai': 'string',
                                                    'weight': 'float',
                                                    'source_bal_pre': 'float',
                                                    'source_bal_post': 'float',
                                                    'target_bal_pre': 'float',
                                                    'target_bal_post': 'float',
                                                    'source': 'int',
                                                    'target': 'int',
                                                    'source_role': 'category',
                                                    'target_role': 'category',
                                                    'right_weight': 'bool',
                                                    'frac_out': 'float',
                                                    'frac_in': 'float',
                                                    'frac_out_post': 'float',
                                                    'frac_in_post': 'float'}
 )

users_fn = '/home/claudio/tesi/sarafu/final_data/03_users_adjusted_allpositivebalances.csv'
USERS = pd.read_csv(users_fn,  # assuming 'user_fn' is your file name
                      sep=",",
                      header=0,
                      dtype={'id': 'int',
                             'start': 'string',
                             'final_bal': 'float',
                             'gender': 'category',
                             'area_name': 'string',
                             'area_type': 'category',
                             'role': 'category',
                             'business': 'category',
                             'poa': 'string',
                             'xdai': 'string',
                             'ovol_in': 'float',
                             'ovol_out': 'float',
                             'otxns_in': 'int',
                             'otxns_out': 'int',
                             'ounique_in': 'int',
                             'ounique_out': 'int',
                             'svol_in': 'float',
                             'svol_out': 'float',
                             'stxns_in': 'int',
                             'stxns_out': 'int',
                             'sunique_in': 'int',
                             'sunique_out': 'int',
                             'crid': 'int',
                             'net': 'float',
                             'initial_bal': 'float',
                             'min_bal': 'float'}) 

TXDF.drop(columns=['Unnamed: 0'],inplace=True)
USERS.drop(columns=['Unnamed: 0'],inplace=True)

formats = ['%Y-%m-%d %H:%M:%S.%f', '%Y-%m-%d %H:%M:%S', '%Y-%m-%d']
for fmt in formats:
    USERS.start = pd.to_datetime(USERS.start, format=fmt, errors='coerce').fillna(USERS.start)
USERS = USERS.sort_values(by='start')

for fmt in formats:
    TXDF.date = pd.to_datetime(TXDF.date, format=fmt, errors='coerce').fillna(TXDF.date)
TXDF = TXDF.sort_values(by='date')

# Define the precision
PRECISION = 3

# Identify columns to exclude (those starting with "frac_")
columns_to_exclude = TXDF.filter(like="frac_").columns

# Identify columns to round (all except those starting with "frac_")
columns_to_round = TXDF.columns.difference(columns_to_exclude)

# Apply rounding only to the selected columns in TXDF
TXDF[columns_to_round] = round_df(TXDF[columns_to_round], PRECISION)

# Apply rounding to all columns in USERS
USERS = round_df(USERS, PRECISION)


In [6]:
END = pd.to_datetime(TXDF.date.max()) #*2021-06-15 05:00:30.887568
START = pd.to_datetime(TXDF.date.min()) #*2020-01-25 19:13:17.731529
NUM_PERIODS = 20
BAD_USERS = [8356,8357,21712,54998]


In [7]:
txdf = TXDF.copy(deep=True)
tx = txdf[(txdf["type"] == "STANDARD") & #keeping just standard transactions
           (txdf["weight"] >= 1.0) & #keeping just transactions with weight >= 1.0
           (txdf["source_role"].isin(["BENEFICIARY", "GROUP_ACCOUNT", "VENDOR"])) & #keeping transactions made by these type of users
           (txdf["target_role"].isin(["BENEFICIARY", "GROUP_ACCOUNT", "VENDOR"]))  #keeping transactions received by these type of users
        ]
agents_outer = basic_agents(tx,USERS.copy(deep=True),how='outer') #create agent dataframe
agents = agents_outer[(agents_outer['tx_in'] > 0) & (agents_outer['tx_out'] > 0)]
filtered_crid = agents.crid.unique() 
tx = tx.loc[(tx.target.isin(filtered_crid))&(tx.source.isin(filtered_crid))]

In [8]:
all_balances = balance_constructor(TXDF[['date','source','target','source_bal_post','target_bal_post','type','id','weight']])

all_balances['period'] = pd.cut(all_balances.date, bins=20,labels=False) #period by date

filtered_balances = all_balances.loc[(all_balances.crid.isin(filtered_crid)) 
                                     & (all_balances.type == 'STANDARD') 
                                     & (all_balances.weight >= 1)]

bal_pivot = pd.pivot_table(filtered_balances, index='crid',columns='period', values='balance', aggfunc='last')
bal_pivot = bal_pivot.ffill(axis=1)
bal_pivot = bal_pivot[bal_pivot>=1]



In [9]:
sarafu = {
        "transactions": tx,
        "agents": agents,
        "balances": all_balances,
        "filtered_balances": filtered_balances,
        "balance_pivot": bal_pivot
    }